# INFO-F-208 Bio-Informatique - Projet 3 - Prédiction de la structure secondaire avec GOR III -- Alexandre Heneffe -- 000440761 -- 17/12/2018
Ce document "jupyter notebook" nous introduit et nous décrit les différents algorithmes implémentés dans le cadre du Mini Projet 3 du cours "Introduction à la bioinformatique"

## Introduction

Dans le cadre du premier projet, nous avons du procéder à l'alignement de séquences d'acides aminés par paire afin de trouver leurs similitudes. Dans le deuxième projet, nous avons poussé cela à l'alignement multiple de séquences afin de représenter celui-ci sous forme de matrice PSSM. Celle-ci représentait un domaine BRD pouvant être visualisé de manière graphique à l'aide des Weblogos.

Pour ce troisième projet, nous allons nous attarder sur la prédiction des structures secondaires (2 dimensions) de séquences protéiques d'acides aminés. En effet, chaque protéine a des chaînes (morceaux de la protéine) ayant des structures secondaires.

Nous voulons donc prédire les structures secondaires de séquences d'acides aminés et les évaluer afin de voir si elles correspondent aux vraies structures. Pour se faire, nous allons utiliser la méthode GOR III pour la prédiction et les mesures Q3 et MCC pour l'évaluation de la qualité des prédictions.

## Matériel

### Fichiers dssp

Nous disposons d'un répertoire contenant des fichiers .dssp représentant chacune des protéines (tirées de WHATIF, un programme utilisé dans une grande variété de domaines de recherche en structure macromoléculaire et permettant d'afficher, manipuler et analyser de petites et grandes molécules et protéines.). Le nom d'un fichier dssp est l'identifiant de la protéine (4 caractères) issue de la base de donnée PDB, un site regroupant un grand nombre de protéines.

DSSP signifie "Define Secondary Structure of Proteins". C'est un programme écrit en PASCAL qui mesure les distances entre atomes pour déterminer les structures secondaires des protéines. Il utilise 7 classes différentes groupées en 3 classes plus générales: Les hélices (G, H et I), les brins (E et B) et les boucles (S, T et C qui est parfois représenté par un espace blanc). Il regarde les liaisons hydrogènes entre les atomes au sein de la chaîne principale.

Ces fichiers .dssp contiennent donc des informations sur les structures secondaires et les probabilités qu'un acide aminé appartienne à une classe H, E ou C. Il permet par exemple de voir que le résidu x est un Valine (V) situé sur la chaine A et appartenant à un brin (E) dans la structure de la protéine.

Comme nous l'avons déjà précisé, nous utiliserons 3 classes de structure secondaire au lieu de 8 :

1. Les symboles H, G et I correspondent à une classe d'hélice (H)
2. Le symbole E et B correspondent à la classe de brin ß (E)
3. Les symboles T, C, S et « espace » correspondent à la classe de coude aléatoire (C)

### CATH_infos

Nous disposons également d'un fichier CATH_infos.txt reprenant les noms des fichiers dssp, ainsi que la chaîne à utiliser pour chacune des protéines. En effet, il se peut qu'un fichier dssp contienne plusieurs chaînes, représentant des copies de la même séquence. 

## Méthode & Implémentation

### Parser

Nous devons implémenter un parser qui va lire chaque fichier dssp précisé par le fichier CATH_infos.txt et récolter les informations contenues dedans afin de construire la protéine et la structure secondaire correspondante. 

Pour chaque fichier dssp, nous aurons comme résultat une protéine avec ce format:
> $>$ identifier|protein name|organism <br>
MTAEPSIVARSNFNVCRLPGTPEAICATYTGSIIIPGATSPGDYAN <br>
CCEECCCCHHHHHHHHHCCCCCHHHHHHHHCCEECCCCCCCHHHCC

Ci-dessous, nous avons la classe représentant le parser DSSP. Celui-ci contient les méthodes nécessaires à la construction des protéines décrites ci-dessus.

In [1]:
class ParserDSSP:
    """
    Classe représentant un parser de fichier .dssp
    """

    def __init__(self, cath_file):
        try:
            self.cath = open(cath_file, "r") # fichier avec les identifiants dssp
        except:
            print("Entrez un fichier valide s.v.p")

        self.secondary_struct = {"H": "H", "G": "H", "I": "H",\
                                 "E": "E", "B": "E"\
                                ,"T": "C", "C": "C", "S": "C", " ": "C"}

    def get_title(self, protein_name, organism):
        """
        Renvoie le titre de la prot sous forme:
        > Identifiant|Nom Prot|Organisme
        """

        res = "> "
        res += protein_name[-2] + "|"      # identifiant
        for n in protein_name[1:-3]:
            res += n + " "
        res = res[:-1] + "|" + organism[0][1:] # organisme
        return res


    def parse(self, dssp_file, chain):
        """
        Parse un fichier dssp et renvoie les informations
        """
        try:
            res = ["> ", "", ""] # contient l'entête de la prot, la prot et structure sec
            d = open(dssp_file, "r")
        except:
            print("fichier dssp invalide")
        
        # ====== TITLE =======
        lines = d.readlines()
        res[0] = self.get_title(lines[2].split(), (lines[4].split(":")[1]).split(';'))

        # ====== PROT + STRUCTURE SECONDAIRE ======
        for i in range(28, len(lines)):
            # Si la chaine est celle qu'on donne et le résidé n'est pas X,Z ou B
            if lines[i][11] == chain and lines[i][13] not in "XZB":
                if lines[i][13].islower():
                    res[1] += "C"
                else:
                    res[1] += lines[i][13] # acide aminé
                res[2] += self.secondary_struct[lines[i][16]] # structure secondaire

        return res[0] + "\n" + res[1] + "\n" + res[2] + "\n"

    def create_protein(self):
        """
        Crée les protéines contenues dans les fichier dssp parsées et les met
        dans un fichier
        """
        try:
            p = open("proteins.txt", "w")
            for line in self.cath:
                l = line.split()[0]
                dssp_file = "dataset/dataset/dssp/" + l[:-1] + ".dssp"
                p.write(self.parse(dssp_file, l[-1]))
        except:
            print("Fichier proteins invalide")

### GOR III

$$I(\triangle S_{j};R_{1}, ..., R_{n}) \approx I(\triangle S_{j};R_{j}) + \sum_{m,m\neq 0} I(\triangle S_{j};R_{j+m}|R_{j})$$

$$I(\triangle S;R) = I(S;R) - I(n-S;R) = log \left( \frac{f_{S,R}}{f_{n-S,R}}\right) + log\left(\frac{f_{n-S}}{f_{s}} \right)$$

$$I(\triangle S_{j};R_{j+m}|R_{j}) = log \left( \frac{f_{S_{j},R_{j+m}, R_{j}}}{f_{n-S_{j}, R_{j+m}, R_{j}}} \right) + log \left( \frac{f_{n-S_{j}, R_{j}}}{f_{S_{j}, R_{j}}} \right) $$

### Statistiques

## Résultats & Discussion

## Conclusion